In [1]:
import re
import os
import sys
import nltk
import pickle
import base64
import time

In [2]:
stemmer = nltk.stem.SnowballStemmer('english')
stop_words = {}
stop_file = open("stop_words.txt", "r")
words = stop_file.read()
words = words.split(",")
for word in words:
    word = word.strip()
    if word:
        stop_words[word[1:-1]] = 1

In [3]:
fields = {}
fields["t"] = open("index/title.txt","r")
fields["c"] = open("index/category.txt","r")
fields["i"] = open("index/infobox.txt","r")
fields["b"] = open("index/body_text.txt","r")
print(fields)

{'t': <_io.TextIOWrapper name='index/title.txt' mode='r' encoding='UTF-8'>, 'c': <_io.TextIOWrapper name='index/category.txt' mode='r' encoding='UTF-8'>, 'i': <_io.TextIOWrapper name='index/infobox.txt' mode='r' encoding='UTF-8'>, 'b': <_io.TextIOWrapper name='index/body_text.txt' mode='r' encoding='UTF-8'>}


In [4]:
words_position = pickle.load(open("index/word_position.pickle", "rb"))
title_position = pickle.load(open("index/title_doc_no.pickle", "rb"))

In [5]:
while True:
    query_result = []
    query = input("> ")
    if (query == "quit") :
        break
    start = time.time()
    query = query.lower().strip()
    if ":" in query:
        query_fields = query.split(",")
        for queries in query_fields:
            field,query = queries.split(":")
            tmp_result = []
            flag = 0
            words = query.split(" ")
            for word in words:
                word = stemmer.stem(word)
                if word and word not in stop_words:
                    if word in words_position and field in words_position[word]:
                        docs = []
                        pointer = words_position[word][field]
                        fields[field].seek(pointer)
                        s = fields[field].readline()[: -1]
                        word , posting_list = s.split("-")
                        posting_list = posting_list.split(",")
                        for doc in posting_list:
                            docs.append(doc.split(":")[0])
                        if flag == 0:
                            tmp_result = docs
                            flag = 1
                        else:
                            tmp_result = list(set(tmp_result) & set(docs))
            for doc_id in tmp_result:
                query_result.append(title_position[int(doc_id)])
    else:
        flag = 0
        tmp_result = []
        query_words = query.split(" ")
        for word in query_words:
            word = stemmer.stem(word)
            if word not in stop_words:
                if word in words_position:
                    docs = []
                    for field in words_position[word].keys():
                        pointer = words_position[word][field]
                        fields[field].seek(pointer)
                        s = fields[field].readline()[: -1]
                        word , posting_list = s.split("-")
                        posting_list = posting_list.split(",")
                        for doc in posting_list:
                            docs.append(doc.split(":")[0])
                    if flag == 0:
                        tmp_result = docs
                        flag = 1
                    else:
                        tmp_result = list(set(tmp_result) & set(docs))
        for doc_id in tmp_result:
            query_result.append(title_position[int(doc_id)])
    
    end = time.time()
    print("Time Taken :- ",(end-start))
    print("No of Results :- ",len(query_result))
    if len(query_result) == 0:
        query_result.append("No result found")
    for result in query_result:
        print(result)
                        

> t:aa
Time Taken :-  3.0279159545898438e-05
No of Results :-  0
No result found
> c:aa
Time Taken :-  0.00011229515075683594
No of Results :-  1
Rolston James
> t:aaa
Time Taken :-  0.0001590251922607422
No of Results :-  1
WCW/AAA When Worlds Collide
> t:aaa ab
Time Taken :-  0.0001742839813232422
No of Results :-  0
No result found
> t:aaa,c:aa
Time Taken :-  0.0002346038818359375
No of Results :-  2
WCW/AAA When Worlds Collide
Rolston James


KeyboardInterrupt: 